In [1]:
import warnings
warnings.filterwarnings('ignore')

# Prepare the Dataset

In [2]:
dataset_name = 'MNIST'

In [3]:
import numpy as np
from keras.datasets import mnist
(X, y), (X_test, y_test) = mnist.load_data()
X = np.concatenate((X, X_test))
y = np.concatenate((y, y_test))
imgs = X
del X_test
del y_test

print('Dataset size {}'.format(X.shape))


Using TensorFlow backend.


Dataset size (70000, 28, 28)


# VAE-CNN

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from utils.constants import Models as models
from models.AE import AE


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [6]:
ae = AE(model_type=models.VAE, dataset_name=dataset_name,hidden_dim=500, plot=True, isConv=True)

Unable to load previous configuration ...
Saving Model Arguments ...


In [7]:
ae.fit(X,y)


Processing data...
Scaling dataset ... 
Creating dask array ... 
Preparing samples ...


Mixture of types in 'arrays'. Falling back to scikit-learn.


Training dataset shape:  (56000, 28, 28, 3)
Validation dataset shape:  (14000, 28, 28, 3)
Sample dataset shape:  (200, 28, 28, 3)
building VAE graph ... 
Instructions for updating:
Colocations handled automatically by placer.

[*] Defining encoders...
Instructions for updating:
Use keras.layers.conv2d instead.
[*] Layer ( encoder_mean/enmean__conv_1/Relu:0 ) output shape: [64, 14, 14, 32]
Instructions for updating:
Use keras.layers.batch_normalization instead.
[*] Layer ( encoder_mean/enmean__conv_2/Relu:0 ) output shape: [64, 7, 7, 32]
[*] Layer ( encoder_mean/enmean__conv_3/Relu:0 ) output shape: [64, 4, 4, 32]
[*] Layer ( encoder_mean/enmean__conv_4/Relu:0 ) output shape: [64, 2, 2, 32]
[*] Layer ( encoder_mean/enmean__conv_5/Relu:0 ) output shape: [64, 1, 1, 32]
Instructions for updating:
Use keras.layers.flatten instead.

Instructions for updating:
Use keras.layers.dense instead.
[*] Layer ( encoder_mean/enmean_dense_1/Relu:0 ) output shape: [64, 500]
Instructions for updating:
Us

ValueError: Variable encoder_mean/batch_norm_1/gamma already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\ProgramData\Anaconda34\envs\gpu_env\lib\site-packages\tensorflow\python\framework\ops.py", line 1801, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\ProgramData\Anaconda34\envs\gpu_env\lib\site-packages\tensorflow\python\framework\ops.py", line 3300, in create_op
    op_def=op_def)
  File "C:\ProgramData\Anaconda34\envs\gpu_env\lib\site-packages\tensorflow\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)


In [11]:
import tensorflow as tf

In [14]:
from networks.dense_net import DenseNet

In [21]:
x_batch = tf.placeholder(tf.float32, [64, 28, 28, 3], name='x_batch')
x_batch_flat = tf.reshape(x_batch , [-1,784])


In [22]:
net = DenseNet(input_=x_batch_flat,
                            hidden_dim=100,
                            output_dim=10,
                            num_layers=3,
                            transfer_fct=tf.nn.relu,
                            act_out=None,
                            reuse=False,
                            kinit=tf.contrib.layers.xavier_initializer(),
                            bias_init=tf.constant_initializer(0.0),
                            drop_rate=0.2,
                            prefix='enmean_')


[*] Layer ( enmean_dense_1_2/Relu:0 ) output shape: [192, 100]
[*] Layer ( enmean_dense_1_dropout/Identity:0 ) output shape: [192, 100]
[*] Layer ( enmean_dense_2/Relu:0 ) output shape: [192, 100]
[*] Layer ( enmean_dense_2_dropout/Identity:0 ) output shape: [192, 100]
[*] Layer ( enmean_H2_batch_norm/batchnorm/add_1:0 ) output shape: [192, 100]
[*] Layer ( enmean_dense_3/BiasAdd:0 ) output shape: [192, 10]


In [28]:
net.__dict__

{'hidden_dim': 100,
 'output_dim': 10,
 'num_layers': 3,
 'transfer_fct': <function tensorflow.python.ops.gen_nn_ops.relu(features, name=None)>,
 'act_out': None,
 'reuse': False,
 'drop_rate': 0.2,
 'prefix': 'enmean_',
 'batch_norm': True,
 'kinit': <function tensorflow.contrib.layers.python.layers.initializers.variance_scaling_initializer.<locals>._initializer(shape, dtype=tf.float32, partition_info=None)>,
 'bias_init': <tensorflow.python.ops.init_ops.Constant at 0x285d0e4d5f8>,
 'output': <tf.Tensor 'enmean_dense_3/BiasAdd:0' shape=(192, 10) dtype=float32>}